In [1]:
import numpy as np
from scipy.linalg import svd

def one_segment(P, w):
    n,d=P.shape
    d-=1

    X = np.zeros((n, d+2))
    for i in range(n):
        ti = P[i, 0]
        pi = P[i, 1:]
        X[i] = np.sqrt(w[i]) * np.concatenate(([1, ti], pi))

    U, Sigma, VT = svd(X, full_matrices=False)
    u = Sigma[0] * VT[0, :]

    c = np.linalg.norm(u) ** 2 / (d + 2)

    Z = np.eye(d + 2)
    Z[:, 0] = u / np.linalg.norm(u) * np.sqrt(c)

    for j in range(1, d+2):
        Z[:, j] = np.sqrt(c) * VT[j-1, :]

    B = Z[:, 1:] / np.sqrt(c)

    C = np.vstack((u, B))
    
    u_weights = np.full(C.shape[0], c)
    
    return C, u_weights

# Example usage
P = np.array([
    [0.1, 1.0, 2.0],
    [0.2, 2.0, 3.0],
    [0.3, 3.0, 4.0],
    [0.4, 4.0, 5.0]
])
w = np.array([1.0, 0.8, 0.6, 0.4])

C, u = one_segment(P, w)
print("Coreset C:\n", C)
print("Weights u:\n", u)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 4 and the array at index 1 has size 3

In [7]:
import numpy as np
from scipy.linalg import svd

def one_segment(P, w):
    n,d=P.shape
    d-=1

    X = np.zeros((n, d+2))
    for i in range(n):
        t_i = P[i, 0]
        p_i = P[i, 1:]
        sqrt_w = np.sqrt(w[i])
        X[i] = sqrt_w * np.hstack(([1], [t_i], p_i))

    U, Sigma, VT = np.linalg.svd(X, full_matrices=False)
    Sigma_VT = np.diag(Sigma) @ VT
    u = Sigma_VT[:, 0]

    c = (np.linalg.norm(u)**2) / (d+2)
    v = np.full(d+2, np.sqrt(c))
    A = np.eye(d+2)
    A[:, 0] = u 
    Q, _ = np.linalg.qr(A)
    D = np.diag(v / (Q[:, 0] @ u))
    Z = Q @ D


    Z_Sigma_VT = Z @ np.diag(Sigma) @ VT
    Z_Sigma_VT_scaled = Z_Sigma_VT / np.sqrt(c)
    d_plus_2 = Z.shape[0]
    B = Z_Sigma_VT_scaled[:, -d_plus_2+1:]

    C = np.vstack([B.T, u])
    
    u_weights = np.full(C.shape[0], c)
    
    return C, u_weights

P = np.array([
    [0.1, 1.0, 2.0],
    [0.2, 2.0, 3.0],
    [0.3, 3.0, 4.0],
    [0.3, 3.0, 4.0],
    [0.3, 3.0, 4.0],
    [0.3, 3.0, 4.0],
    [0.4, 4.0, 5.0]
])
w = np.array([1.0, 0.8, 0.5, 1.0, 0.7, 0.6, 0.4])

C, u = one_segment(P, w)
print("Coreset C:\n", C)
print("Weights u:\n", u)


Coreset C:
 [[ 2.62363425e-01 -5.25873880e-02  1.57655013e-18  3.12369644e-17]
 [ 2.62363425e+00 -5.25873880e-01 -3.01030411e-16 -1.05098181e-17]
 [ 3.48095110e+00 -1.04066297e+00 -2.09595455e-16 -1.94689127e-17]
 [-2.15482996e+00  5.97250250e-01  1.79416765e-16  9.43887057e-18]]
Weights u:
 [1.25 1.25 1.25 1.25]
